In [341]:
import Functions as fn

In [2]:
import importlib

In [1]:
import pandas as pd
import numpy as np

In [2]:
import time

In [3]:
try:
    importlib.reload(fn)
except:
    import function as fn

<module 'functionsv1' from 'C:\\Users\\Chirag\\Machine learning\\Version1(oct2021)\\functionsv1.py'>

In [124]:
prices = pd.read_csv("simfin_data/us-shareprices-daily.csv", sep = ";")
prices = prices.set_index(["Ticker"])

In [66]:
income = pd.read_csv("simfin_data/us-income-quarterly-full-asreported.csv", sep = ";")
income = income.set_index(["Ticker"])

In [67]:
balance = pd.read_csv("simfin_data/us-balance-quarterly-full-asreported.csv", sep = ";")
balance = balance.set_index(["Ticker"])

In [70]:
cashflow = pd.read_csv("simfin_data/us-cashflow-quarterly-full-asreported.csv", sep = ";")
cashflow = cashflow.set_index(["Ticker"])

In [68]:
derived_figures = pd.read_csv("simfin_data/us-derived-quarterly-asreported.csv", sep = ";")
derived_figures = derived_figures.set_index(["Ticker"])

In [69]:
derived_shareprices = pd.read_csv("simfin_data/us-derived-shareprices-daily-asreported.csv", sep = ";")
derived_shareprices = derived_shareprices.set_index(["Ticker"])

In [71]:
len(income), len(balance), len(derived_figures), len(cashflow)

(79417, 79417, 79417, 79417)

In [109]:
len(prices), len(derived_shareprices)

(7693639, 7693639)

In [125]:
Tickers_income = np.unique(income.index)
Tickers_balance = np.unique(balance.index)
Tickers_cashflow = np.unique(cashflow.index)
Tickers_derived_figures = np.unique(derived_figures.index)
Tickers_prices = np.unique(prices.index)
Tickers_derived_shareprices = np.unique(derived_shareprices.index)

In [74]:
len(Tickers_income), len(Tickers_balance), len(Tickers_cashflow), len(Tickers_derived_figures)

(2367, 2367, 2367, 2367)

In [126]:
len(Tickers_prices), len(Tickers_derived_shareprices)

(2711, 2711)

In [181]:
Tickers_intersect = np.intersect1d(Tickers_prices, Tickers_income)

In [182]:
len(Tickers_intersect)

2317

In [183]:
"A" in Tickers_intersect

True

In [184]:
"""
get the list of dates by Dates columns for a well known stock
"""
Dates = prices.loc["A"]["Date"].values
print(len(Dates))
print(Dates[0])
print(Dates[-1])


2600
2011-05-20
2021-09-17


In [185]:
"""
we want last 10 years of data for modelling, it cant hurt, and I want to see if the models hold up in a completley different environment
"""
Date = Dates[-2600:]

In [186]:
first_date = Date[0]
last_date = Date[-1]
first_date, last_date

('2011-05-20', '2021-09-17')

In [129]:
prices = prices.reset_index().set_index(["Date"]).loc[Date]

In [130]:
prices  = prices.reset_index().set_index(['Ticker']).loc[Tickers_intersect]

In [187]:
"""
removing tickers that have an underscore in the name, a naming convention by the data provider to identify stocks that are no monger trading or something
"""
Tickers_test = []
for i in Tickers_intersect:
    if len(i.split("_")) ==1:
        Tickers_test.append(i)
    else:
        print(i)
        
print(len(Tickers_intersect) - len(Tickers_test) )
Tickers_intersect = Tickers_test

ACET_delis
ADPT_delis
AGN_old
ARMK_old
AVGO_old
ENDP_old
ESI_delist
ES_old
FOX_old
FTI_old
GGP_old
LDOS_old
LSI_old
MYL_old
WFT_old
WMGI_delis
16


In [190]:
del start_end_index

In [193]:
Tickers_continuity = []
Tickers_bad = []
Tickers_error = []
start_end_list = []
start_time = time.time()
for i in Tickers_intersect:
    try:
        if np.sum(prices.loc[i]["Close"].isna()) > 0:
            Tickers_bad.append(i)
            
        else:
            start = prices.loc[i].iloc[0]["Date"]
            end = prices.loc[i].iloc[-1]["Date"]
            length_acc = int(len(prices.loc[i]["Close"]))
            start_indx = np.where(Date == start)[0][0]
            end_indx = np.where(Date == end)[0][0]
            length_proper = int(end_indx - start_indx + 1)
            if length_acc == length_proper:
                Tickers_continuity.append(i)
                start_end_list.append(start)
                start_end_list.append(end)
            else:
                Tickers_bad.append(i)
    except:
        Tickers_error.append(i)
        

        
print(time.time() - start_time)

1.513340711593628


In [194]:

"""
so there were over 100 stocks that didnt have any nan data but were missing data points entirely
the only stock that had an error was "WIN", because it only has one data point lmao

"""
len(Tickers_continuity)

2156

In [195]:
Tickers_intersect = Tickers_continuity

In [196]:
start_end_list= np.reshape(start_end_list, (-1, 2))

In [ ]:
"""
just need to make sure that the income and balance and other sheets are consistent in that there are no missing data points between the stock start date and end date for close prices


"""

In [199]:
income = income.loc[Tickers_intersect]
balance = balance.loc[Tickers_intersect]
cashflow = cashflow.loc[Tickers_intersect]
derived_figures = derived_figures.loc[Tickers_intersect]


In [344]:
def check(x):
    
    for i in range(len(x)):
        
        
        if i == len(x)-1:
            return 1
        elif x[i] == "Q1" and x[i+1] != "Q2":
            return 0
        elif x[i] == "Q2" and x[i+1] != "Q3":
            return 0
        elif x[i] == "Q3" and x[i+1] != "Q4":
            return 0
        elif x[i] == "Q4" and x[i+1] !="Q1":
            return 0
        else:
            continue

In [346]:
Tickers_good = []
Tickers_bad = []
Tickers_error = []
for i in Tickers_intersect:
    
    try:
        if len(income.loc[i]) >= 5:
            if check(income.loc[i, "Fiscal Period"].values):
                Tickers_good.append(i)
            else:
                Tickers_bad.append(i)
        else:
            Tickers_bad.append(i)
        
        
    except:
        Tickers_error.append(i)

In [350]:
len(Tickers_good)

2077

In [ ]:
"""
21 stocks had errors because they only had one entry
58 stocks did not fit the criteria
96% of stocks met the criteria
"""

In [352]:
Tickers_intersect = Tickers_good

In [358]:
prices = prices.loc[Tickers_intersect]

In [23]:
Tickers_good = []
Tickers_bad = []
for i in Tickers_intersect:
    if len(prices.loc[i]) <100:
        Tickers_bad.append(i)
    else:
        Tickers_good.append(i)
        
Tickers_intersect = Tickers_good

"""
there were only like 10 stocks that had less than 100 data points
"""

In [361]:
derived_shareprices = derived_shareprices.reset_index().set_index(["Date"]).loc[Date]
derived_shareprices  = derived_shareprices.reset_index().set_index(['Ticker']).loc[Tickers_intersect]

In [354]:
income = income.loc[Tickers_intersect]
balance = balance.loc[Tickers_intersect]
cashflow = cashflow.loc[Tickers_intersect]
derived_figures = derived_figures.loc[Tickers_intersect]

In [ ]:
balance["Working Capital"] = balance["Total Current Assets"] - balance["Total Current Liabilities"]

In [46]:
income.to_pickle("parsed_data/income")
balance.to_pickle("parsed_data/balance")
cashflow.to_pickle("parsed_data/cashflow")
derived_figures.to_pickle("parsed_data/derived_figures")

In [48]:
prices.to_pickle("parsed_data/prices")
derived_shareprices.to_pickle("parsed_data/derived_shareprices")

In [393]:
"""pickle speed to read data is 17% of csv"""

17.642726304240412